# Mock Classifier With Custom Endpoints Model

 * Wrap a basic python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
   * Example of using custom endpoints that are scraped by prometheus
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install grpcio-tools
```
 

Wrap model using s2i

## Test locally using REST

In [ ]:
!make build_rest

In [ ]:
!docker run --name "mock_classifier_with_custom_endpoints_rest" -d --rm \
    -e PREDICTIVE_UNIT_SERVICE_PORT=5000 \
    -p 5000:5000 -p 5055:5055 seldonio/mock_classifier_with_custom_endpoints_rest:1.0

In [ ]:
!cd ../../../wrappers/testing && make build_protos

In [ ]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract
#
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

In [ ]:
#
# Call the custom endpoint.
# In this example its used for the prediction call count.
#
!curl "http://localhost:5055/prometheus_metrics"

In [ ]:
!docker rm -v "mock_classifier_with_custom_endpoints_rest" --force

## Test locally using GRPC

In [ ]:
!make build_grpc

In [ ]:
!docker run --name "mock_classifier_with_custom_endpoints_rest" -d --rm \
    -e PREDICTIVE_UNIT_SERVICE_PORT=5000 \
    -p 5000:5000 -p 5055:5055 seldonio/mock_classifier_with_custom_endpoints_grpc:1.0

In [ ]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract,
# using NDArray.
#
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

In [ ]:
#
# Call the prediction endpoint.
# Send some random features that conform to the contract,
# using Tensor.
#
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc --tensor

In [ ]:
#
# Call the custom endpoint.
# In this example its used for the prediction call count.
#
!curl "http://localhost:5055/prometheus_metrics"

In [ ]:
!docker rm -v "mock_classifier_with_custom_endpoints_rest" --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:

!minikube start --memory=8096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
#
# Create a cluster-admin role binding
#
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
#
# Start seldon-core and analytics
#
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set prometheus.service_type=NodePort

In [ ]:
!eval $(minikube docker-env) && make build_rest

In [ ]:
#
# Create moldel deployment
#
!kubectl create -f model-deployment.json

In [ ]:
#
# Wait until ready (replicas == replicasAvailable)
#
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!cd ../../../util/api_tester && make build_protos

In [ ]:
#
# Make prediction request.
#
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
#
# Query prometheus for the custom metrics
#
# A "custom_service" is run on port "5055" that
# is defined in model code "MeanClassifier.py".
#
# Prometeus set to scrape the model pod on port "5055"
# by the annotations in the "model-deployment.json" manifest.
#
!curl -s "$(minikube ip):$(kubectl get svc prometheus-seldon -o jsonpath='{.spec.ports[0].nodePort}')/api/v1/query?query=predict_call_count"

In [ ]:
#
# The prometheus metrics can also been seen in a browser
# using the url shown after running this cell.
#
# In the prometheus ui execute the expression "predict_call_count"
#
!echo "http://$(minikube ip):$(kubectl get svc prometheus-seldon -o jsonpath='{.spec.ports[0].nodePort}')"

In [ ]:
#
# Clean up
#
!minikube delete